In [0]:
%python 
df = spark.sql(f'''
    with integrated_remote_users as (
        SELECT
            date_ym
            , mac_addr
            , normal_log as log_uvc_list -- 통합리모컨 사용자
        FROM kic_data_ods.tlamp.normal_log_webos25
        WHERE 1=1
        AND X_device_country = 'KR'
        AND context_name = 'ICS'
        AND message_id = 'NL_UVC_LIST' -- 통합리모컨 사용자
        AND date_ym >= '2025-09'
        AND date_ym <= '2025-11'
    ), magic_remote_log as (
        select distinct
                date_ym
                , mac_addr
                , first(normal_log) as log_pairing_check
        FROM  kic_data_ods.tlamp.normal_log_webos25
        WHERE 1=1
        AND X_device_country = 'US'
        AND context_name = 'lginput2'
        AND message_id = 'NL_PAIRING_CHECK' -- 매직리모컨 
        AND date_ym >= '2025-09'
        AND date_ym <= '2025-11'
        GROUP BY date_ym, mac_addr
    ), fact_home_shelf as (
        SELECT
            date_ym
            , mac_addr
            , count(1) as cnt_home_shelf
        FROM kic_data_ods.tlamp.normal_log_webos25
        WHERE 1=1
        AND X_device_country = 'US'
        AND context_name = 'com.webos.app.home'
        AND message_id = 'NL_SHELF_LIST_SHOWN' 
        AND date_ym >= '2025-09'
        AND date_ym <= '2025-11'
        group by date_ym, mac_addr
    ), fact_home_home as (
        SELECT
            date_ym
            , mac_addr
            , count(1) as cnt_home_home
        FROM kic_data_ods.tlamp.normal_log_webos25
        WHERE 1=1
        AND X_device_country = 'US'
        AND context_name = 'com.webos.app.home'
        AND message_id = 'NL_HOME_SHOWN' 
        AND date_ym >= '2025-09'
        AND date_ym <= '2025-11'
        group by date_ym, mac_addr
    ), fact_home_hero as (
        SELECT
            date_ym
            , mac_addr
            , count(1) as cnt_home_hero
        FROM kic_data_ods.tlamp.normal_log_webos25
        WHERE 1=1
        AND X_device_country = 'US'
        AND context_name = 'com.webos.app.home'
        AND message_id = 'NL_HERO_SHOWN' 
        AND date_ym >= '2025-09'
        AND date_ym <= '2025-11'
        group by date_ym, mac_addr
    ), fact_home_app as (
        SELECT
            date_ym
            , mac_addr
            , count(1) as cnt_home_app
        FROM kic_data_ods.tlamp.normal_log_webos25
        WHERE 1=1
        AND X_device_country = 'US'
        AND context_name = 'com.webos.app.home'
        AND message_id = 'NL_APP_LAUNCH'
        AND date_ym >= '2025-09'
        AND date_ym <= '2025-11'
        group by date_ym, mac_addr
    ), fact_home_lgchannel as (
        SELECT
            date_ym
            , mac_addr
            , count(1) as cnt_home_lgchannel
        FROM kic_data_ods.tlamp.normal_log_webos25
        WHERE 1=1
        AND X_device_country = 'US'
        AND context_name = 'com.webos.app.livedmost'
        AND message_id = 'NL_LGCHANNELS'
        AND date_ym >= '2025-09'
        AND date_ym <= '2025-11'
        group by date_ym, mac_addr
    )

    select it.date_ym
           , it.mac_addr
           , it.log_uvc_list
           , mg.log_pairing_check
           , fs.cnt_home_shelf
           , fho.cnt_home_home
           , fhe.cnt_home_hero
           , fa.cnt_home_app
           , fl.cnt_home_lgchannel
    from   integrated_remote_users it
    left join magic_remote_log mg using(date_ym, mac_addr)
    left join fact_home_shelf fs using(date_ym, mac_addr)
    left join fact_home_home fho using(date_ym, mac_addr)
    left join fact_home_hero fhe using(date_ym, mac_addr)
    left join fact_home_app fa using(date_ym, mac_addr)
    left join fact_home_lgchannel fl using(date_ym, mac_addr)
''') 


In [0]:
df\
    .write.mode('overwrite')\
    .option("mergeSchema", "true")\
    .saveAsTable(f"sandbox.z_eunmi1_ko.temp_1206")

In [0]:
df_result = spark.sql(r"""
    SELECT
    date_ym
    , mac_addr
    , dev_list.cec_depth as log_uvc_cec_depth
    , dev_list.spd_source_device_info as log_uvc_spd_source_device_info
    , dev_list.spd_product_description as log_uvc_spd_product_description
    , dev_list.powersync_method as log_uvc_powersync_method
    , dev_list.input as log_uvc_input
    , dev_list.count_used as log_uvc_count_used
    , dev_list.dev_name as log_uvc_dev_name
    , dev_list.cec_osd_name as log_uvc_cec_osd_name
    , dev_list.setup_method as log_uvc_setup_method
    , dev_list.setup_done as log_uvc_setup_done
    , dev_list.cec_vendor_id as log_uvc_cec_vendor_id
    , dev_list.dev_type as log_uvc_dev_type
    , dev_list.dev_model as log_uvc_dev_model
    , dev_list.control_method as log_uvc_control_method
    , dev_list.cec_unique_id as log_uvc_cec_unique_id
    , dev_list.cec_physical_address as log_uvc_cec_physical_address
    , dev_list.spd_vendor_name as log_uvc_spd_vendor_name
    , dev_list.count_used_keylist as log_uvc_count_used_keylist
    , log_pairing_check
    , cnt_home_shelf
    , cnt_home_home
    , cnt_home_hero
    , cnt_home_app
    , cnt_home_lgchannel
    FROM
    (
        SELECT
        date_ym
        , mac_addr
        , log_uvc_list
        , log_pairing_check
        , cnt_home_shelf
        , cnt_home_home
        , cnt_home_hero
        , cnt_home_app
        , cnt_home_lgchannel
        FROM
        sandbox.z_eunmi1_ko.temp_1206
    ) t
    LATERAL VIEW OUTER
    explode(
    from_json(
    regexp_replace(log_uvc_list, '\\{\\}\\s', ''),
    'STRUCT<dev_list: ARRAY<STRUCT<cec_depth:STRING,count_used_keylist:STRING,spd_source_device_info:STRING,spd_product_description:STRING,powersync_method:STRING,input:STRING,count_used:STRING,dev_name:STRING,cec_osd_name:STRING,setup_method:STRING,setup_done:STRING,cec_vendor_id:STRING,dev_type:STRING,dev_model:STRING,control_method:STRING,cec_unique_id:STRING,cec_physical_address:STRING,spd_vendor_name:STRING>>>'
    ).dev_list
    ) AS dev_list
""")

df_result.createOrReplaceTempView("df_result")

In [0]:
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_name = notebook_path.split("/")[-1]

df_result.drop('mac_addr').coalesce(1)\
  .write.format("com.databricks.spark.csv")\
  .mode("overwrite").option("header", "true")\
  .save(f's3://s3-lge-he-inbound-kic-dev/HEDS/{notebook_name}')